In [1]:
import numpy as np
import cv2
from pathlib import Path
import pandas as pd

In [2]:
from itertools import chain
from shutil import copyfile
from pprint import pprint
from tqdm.notebook import tqdm
from collections import defaultdict
from pdb import set_trace
from typing import *
from functools import partial

In [3]:
import imageio
import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

In [4]:
%cd ..
from my_utils.general import *

/notebooks/nbs/yolov5


In [5]:
%ls my_utils

__pycache__/  copy_by_df.py  dataset_util.py  dict_to_df.py  general.py


In [6]:
%load_ext autoreload
%autoreload 2

from my_utils.dataset_util import images_from_l3_dir__top_1img
import my_utils.dict_to_df as dtdf
# from my_utils.copy_by_df import create_img_lbl_dest, train_df_to_image_label

In [7]:
def set_path(is_kaggle):
    
    if is_kaggle:
        kaggle = Path('/kaggle')

    else:
        kaggle = Path('/storage/kaggle')

    in_path = kaggle/'input'
    out_path = kaggle/'working'
    
    return kaggle, in_path, out_path

In [8]:
kaggle, in_path, out_path = set_path(False)

In [9]:
product_id_dict = {
    'hundred_plus': 0,
    'kikapoo': 1,
    'cincau': 2,
    'milo_nuggets': 3,
    'mimi': 4,
    'maggi': 5,
    'soya': 6,
    'jvhua': 7,
    'nescafe': 8,
    'himalaya': 9
}

In [17]:
def load_util(filename):
    f_path = f"my_utils/{filename}"
    %load {f_path}


# start process

In [18]:
kaggle, in_path, out_path = set_path(False)

In [19]:
s1_path = in_path/'ten-pd-spin-1'
f_path = in_path/'ten_products_fridge'

In [20]:
ls(s1_path)

[PosixPath('/storage/kaggle/input/ten-pd-spin-1/predict_labels'),
 PosixPath('/storage/kaggle/input/ten-pd-spin-1/images')]

In [21]:
top1_s1_imgs = images_from_l3_dir__top_1img(s1_path/'images'/'images', 1)
top1_s1_lbls = {pd.name: ls(pd) for pd in ls(s1_path/'predict_labels'/'predict_labels')}

In [22]:
f_imgs = {pd.name: ls(pd) for pd in ls(f_path/'images')}
f_lbls = {pd.name: ls(pd) for pd in ls(f_path/'draw_and_predict_label') if pd.name != 'labels'}

In [23]:
rename_keys = {
    'kickapoo': 'kikapoo',
    '100Plus': 'hundred_plus',
    'milonugget': 'milo_nuggets',
    'nutriwellcincau': 'cincau',
    'hot_cup_maggi': 'maggi',
    'himalaya_salt_candy': 'himalaya',
    'chrysanthemum': 'jvhua'
}

top1_s1_imgs = rename_dict(top1_s1_imgs, rename_keys)

In [24]:
rename_keys = {
    'kickapoo': 'kikapoo',
    '100Plus': 'hundred_plus',
    'MiloNuggets': 'milo_nuggets',
    'nutriwellcincau': 'cincau',
}

f_imgs = rename_dict(f_imgs, rename_keys)

In [25]:
ls(kaggle/'input'/'ten-pd-spin-1')

[PosixPath('/storage/kaggle/input/ten-pd-spin-1/predict_labels'),
 PosixPath('/storage/kaggle/input/ten-pd-spin-1/images')]

In [123]:
print('top1_s1')
print('-'*15)
pd_len_dict([top1_s1_imgs, top1_s1_lbls])
print('fridge')
print('-'*15)
pd_len_dict([f_imgs, f_lbls])

top1_s1
---------------


,cincau,himalaya,hundred_plus,jvhua,kikapoo,maggi,milo_nuggets,mimi,nescafe,soya
0,161,161,162,161,161,161,161,161,161,161
1,200,195,201,200,200,200,200,200,200,200


fridge
---------------


,cincau,himalaya,hundred_plus,jvhua,kikapoo,maggi,milo_nuggets,mimi,nescafe,soya
0,400,800,400,800,400,800,400,800,800,800
1,371,672,395,777,370,777,399,784,754,767


# dict to df

In [28]:
yolo_csv_column = ['class_id', 'x_center', 'y_center', 'width', 'height']
column_name = [*yolo_csv_column, 'file_name']

In [124]:
f_imgs_df = dtdf.image_dict_to_df(f_imgs)
f_lbls_df = {class_name: dtdf.label_list_toDf(lbl_list, column_name, class_name, product_id_dict) 
                  for class_name, lbl_list in f_lbls.items()}

In [125]:
top1_s1_imgs_df = dtdf.image_dict_to_df(top1_s1_imgs)
top1_s1_lbls_df = {class_name: dtdf.label_list_toDf(lbl_list, column_name, class_name, product_id_dict) 
                  for class_name, lbl_list in top1_s1_lbls.items()}

In [131]:
if is_key_match(top1_s1_imgs_df, top1_s1_lbls_df):
    top1_s1_train_joined_df = dtdf.join_img_and_lbl_df(top1_s1_imgs_df, top1_s1_lbls_df)
else:
    print('key not match!')
    

if is_key_match(f_imgs_df, f_lbls_df):
    f_val_joined_df = dtdf.join_img_and_lbl_df(f_imgs_df, f_lbls_df)
else:
    print('key not match!')
    
    

In [132]:
pd_len_dict([top1_s1_imgs_df, top1_s1_lbls_df, top1_s1_train_joined_df])

print('fridge')
print('-'*25)
pd_len_dict([f_imgs_df, f_lbls_df, f_val_joined_df])

,cincau,himalaya,hundred_plus,jvhua,kikapoo,maggi,milo_nuggets,mimi,nescafe,soya
0,161,161,162,161,161,161,161,161,161,161
1,200,195,201,200,200,200,200,200,200,200
2,161,156,162,161,161,161,161,161,161,161


fridge
-------------------------


,cincau,himalaya,hundred_plus,jvhua,kikapoo,maggi,milo_nuggets,mimi,nescafe,soya
0,400,800,400,800,400,800,400,800,800,800
1,371,672,395,778,370,777,399,784,754,767
2,371,672,395,778,370,777,399,784,754,767


# copy img and save label

In [32]:
import my_utils.copy_by_df as cp_df

In [33]:
images_dest = kaggle/'working/top1_s1/images'
label_dest = kaggle/'working/top1_s1/labels'

cp_df.create_img_lbl_dest(images_dest, label_dest)
ls(out_path)

[PosixPath('/storage/kaggle/working/upload'),
 PosixPath('/storage/kaggle/working/runs'),
 PosixPath('/storage/kaggle/working/top1_s1')]

In [34]:
'''write label and copy images for (train top1_s1)'''

for k, df in tqdm(top1_s1_train_joined_df.items()):
    cp_df.train_df_to_image_label(df, label_dest, images_dest)

In [133]:
for k, df in tqdm(f_val_joined_df.items()):
    cp_df.val_df_to_image_label(df, label_dest, images_dest, sample_size=0.3)
    

In [128]:
cp_df.len_dest('train', images_dest, label_dest)

1625
match


In [134]:
cp_df.len_dest('val', images_dest, label_dest)

1819
match
